<h1 style="text-align: center; font-family: Verdana; font-size: 32px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; font-variant: small-caps; letter-spacing: 3px; color: #fb806f; background-color: #ffffff;">G-Research Crypto Forecasting</h1>
<h2 style="text-align: center; font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: underline; text-transform: none; letter-spacing: 2px; color: black; background-color: #ffffff;">Exploratory Data Analysis</h2>
<h5 style="text-align: center; font-family: Verdana; font-size: 12px; font-style: normal; font-weight: bold; text-decoration: None; text-transform: none; letter-spacing: 1px; color: black; background-color: #ffffff;">CREATED BY: DARIEN SCHETTLER</h5>

<br>

---

<br>

<div class="alert alert-block alert-warning" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">📖 &nbsp; I USED THE FOLLOWING NOTEBOOKS... PLEASE UPVOTE THEM AS WELL!</b><br><br>
    - <a style="color: black;", href="https://www.kaggle.com/carlmcbrideellis/g-research-plot-financial-data-using-mplfinance"><b>G-Research: Plot financial data using mplfinanc</b></a><br>
</div>

<br>

<font size=18px, color="red"><center><b>VERY WIP</b></center></font>


<p id="toc"></p>

<br><br>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: #fb806f; background-color: #ffffff;">TABLE OF CONTENTS</h1>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#imports">0&nbsp;&nbsp;&nbsp;&nbsp;IMPORTS</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#background_information">1&nbsp;&nbsp;&nbsp;&nbsp;BACKGROUND INFORMATION</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#setup">2&nbsp;&nbsp;&nbsp;&nbsp;SETUP</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#helper_functions">3&nbsp;&nbsp;&nbsp;&nbsp;HELPER FUNCTIONS</a></h3>

---

<br>

<a id="imports"></a>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: #fb806f;" id="imports">0&nbsp;&nbsp;IMPORTS&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a></h1>

In [ ]:
print("\n... IMPORTS STARTING ...\n")
print("\n\tVERSION INFORMATION")
# Machine Learning and Data Science Imports
import tensorflow as tf; print(f"\t\t– TENSORFLOW VERSION: {tf.__version__}");
import tensorflow_addons as tfa; print(f"\t\t– TENSORFLOW ADDONS VERSION: {tfa.__version__}");
import pandas as pd; pd.options.mode.chained_assignment = None;
import numpy as np; print(f"\t\t– NUMPY VERSION: {np.__version__}");
import sklearn; print(f"\t\t– SKLEARN VERSION: {sklearn.__version__}");
from sklearn.preprocessing import RobustScaler, PolynomialFeatures
from sklearn.model_selection import GroupKFold;

# Competition Specific Imports
import gresearch_crypto
import cudf, cuml, cupy

!pip -q install mplfinance
!pip -q install chart_studio
import mplfinance as mpf
import cufflinks as cf; cf.set_config_file(offline=True)

# Built In Imports
from kaggle_datasets import KaggleDatasets
from collections import Counter
from datetime import datetime
from glob import glob
import warnings
import requests
import imageio
import IPython
import sklearn
import urllib
import zipfile
import pickle
import random
import shutil
import string
import math
import time
import gzip
import ast
import sys
import io
import os
import gc
import re

# Visualization Imports
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm; tqdm.pandas();
import plotly.express as px
import seaborn as sns
from PIL import Image
import matplotlib; print(f"\t\t– MATPLOTLIB VERSION: {matplotlib.__version__}");
import plotly
import PIL
import cv2


def seed_it_all(seed=7):
    """ Attempt to be Reproducible """
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    
print("\n\n... IMPORTS COMPLETE ...\n")
    
print("\n... SEEDING FOR DETERMINISTIC BEHAVIOUR ...\n")
seed_it_all()

<br>

<a id="background_information"></a>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: #fb806f; background-color: #ffffff;" id="background_information">1&nbsp;&nbsp;BACKGROUND INFORMATION&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a></h1>

<br><b style="text-decoration: underline; font-family: Verdana; text-transform: uppercase;">PRIMARY TASK DESCRIPTION</b>


**In this competition you will forecast the prices of several cyrptoassets. Once you make that prediction, you can move on to the next batch and will get additional data.**

This competition is different from most Kaggle Competitions in that:
* You can only submit from Kaggle Notebooks
* You must use our custom **`gresearch_crypto`** Python module.  
    * The purpose of this module is to control the flow of information to ensure that you are not using future data to make predictions.  
    * If you do not use this module properly, your code may fail.


---

<br>

<br><b style="text-decoration: underline; font-family: Verdana; text-transform: uppercase;">INTERACTING WITH THE TEST API</b>

You must leverage an environment from a python package and iterate through using the **`iter_test`** function:

<b><code><mark>iter_test</mark></code></b>
* This is a generator which loops through each timestamp in the test set. 
* You have direct access to the example test rows for your convenience, but your code will only be able to get rows from the real test set via the API. 
* Once you call **`predict`** you can continue on to the next batch.

**What Does It Yield**
* While there are more batch(es) and `predict` was called successfully since the last yield, yields a tuple of:
    * **`test_df`**: DataFrame with the test features for the next batch, and user responses for the previous batch.
    * **`sample_prediction_df`**: DataFrame with an example prediction.  
    * Intended to be filled in and passed back to the `predict` function.
* If **`predict`** has not been called successfully since the last yield, prints an error and yields `None`.

<br>

---

<br>


<a id="setup"></a>


<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: #fb806f; background-color: #ffffff;"  id="setup">2&nbsp;&nbsp;SETUP&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a>
</h1>

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #fb806f; background-color: #ffffff;">2.1 ACCELERATOR DETECTION</h3>

---

In order to use **`TPU`**, we use **`TPUClusterResolver`** for the initialization which is necessary to connect to the remote cluster and initialize cloud TPUs. Let's go over two important points

1. When using TPU on Kaggle, you don't need to specify arguments for **`TPUClusterResolver`**
2. However, on **G**oogle **C**ompute **E**ngine (**GCE**), you will need to do the following:

<br>

```python
# The name you gave to the TPU to use
TPU_WORKER = 'my-tpu-name'

# or you can also specify the grpc path directly
# TPU_WORKER = 'grpc://xxx.xxx.xxx.xxx:8470'

# The zone you chose when you created the TPU to use on GCP.
ZONE = 'us-east1-b'

# The name of the GCP project where you created the TPU to use on GCP.
PROJECT = 'my-tpu-project'

tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=TPU_WORKER, zone=ZONE, project=PROJECT)
```

<div class="alert alert-block alert-danger" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">🛑 &nbsp; WARNING:</b><br><br>- Although the Tensorflow documentation says it is the <b>project name</b> that should be provided for the argument <b><code>`project`</code></b>, it is actually the <b>Project ID</b>, that you should provide. This can be found on the GCP project dashboard page.<br>
</div>

<div class="alert alert-block alert-info" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">📖 &nbsp; REFERENCES:</b><br><br>
    - <a href="https://www.tensorflow.org/guide/tpu#tpu_initialization"><b>Guide - Use TPUs</b></a><br>
    - <a href="https://www.tensorflow.org/api_docs/python/tf/distribute/cluster_resolver/TPUClusterResolver"><b>Doc - TPUClusterResolver</b></a><br>

</div>

In [ ]:
print(f"\n... ACCELERATOR SETUP STARTING ...\n")

# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    TPU = tf.distribute.cluster_resolver.TPUClusterResolver()  
except ValueError:
    TPU = None

if TPU:
    print(f"\n... RUNNING ON TPU - {TPU.master()}...")
    tf.config.experimental_connect_to_cluster(TPU)
    tf.tpu.experimental.initialize_tpu_system(TPU)
    strategy = tf.distribute.experimental.TPUStrategy(TPU)
else:
    print(f"\n... RUNNING ON CPU/GPU ...")
    # Yield the default distribution strategy in Tensorflow
    #   --> Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy() 

# What Is a Replica?
#    --> A single Cloud TPU device consists of FOUR chips, each of which has TWO TPU cores. 
#    --> Therefore, for efficient utilization of Cloud TPU, a program should make use of each of the EIGHT (4x2) cores. 
#    --> Each replica is essentially a copy of the training graph that is run on each core and 
#        trains a mini-batch containing 1/8th of the overall batch size
N_REPLICAS = strategy.num_replicas_in_sync
    
print(f"... # OF REPLICAS: {N_REPLICAS} ...\n")

print(f"\n... ACCELERATOR SETUP COMPLTED ...\n")

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #fb806f; background-color: #ffffff;">2.2 COMPETITION DATA ACCESS</h3>

---

TPUs read data must be read directly from **G**oogle **C**loud **S**torage **(GCS)**. Kaggle provides a utility library – **`KaggleDatasets`** – which has a utility function **`.get_gcs_path`** that will allow us to access the location of our input datasets within **GCS**.<br><br>

<div class="alert alert-block alert-info" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">📌 &nbsp; TIPS:</b><br><br>- If you have multiple datasets attached to the notebook, you should pass the name of a specific dataset to the <b><code>`get_gcs_path()`</code></b> function. <i>In our case, the name of the dataset is the name of the directory the dataset is mounted within.</i><br><br>
</div>

In [ ]:
print("\n... DATA ACCESS SETUP STARTED ...\n")

if TPU:
    # Google Cloud Dataset path to training and validation images
    DATA_DIR = KaggleDatasets().get_gcs_path('g-research-crypto-forecasting')
    save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
else:
    # Local path to training and validation images
    DATA_DIR = "/kaggle/input/g-research-crypto-forecasting"
    save_locally = None
    
print(f"\n... DATA DIRECTORY PATH IS:\n\t--> {DATA_DIR}")

print(f"\n... IMMEDIATE CONTENTS OF DATA DIRECTORY IS:")
for file in tf.io.gfile.glob(os.path.join(DATA_DIR, "*")): print(f"\t--> {file}")

    
print("\n\n... DATA ACCESS SETUP COMPLETED ...\n")

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #fb806f; background-color: #ffffff;">2.3 BASIC DATA DEFINITIONS & INITIALIZATIONS</h3>

---

Let's import the module and create an environment. 
* Adding the directory holding the module to the pythonpath with **`sys.path.append`** isn't strictly necessary within Kaggle notebooks, which handles that behind the scenes, but will be necessary if you are testing your code offline.
* You can only call make_env() **once**, so don't lose it!

In [ ]:
print("\n... BASIC DATA SETUP STARTING ...\n\n")

print("\n... ASSET DETAILS DATAFRAME ..\n")
ASSET_CSV = os.path.join(DATA_DIR, "asset_details.csv")
asset_df = cudf.read_csv(ASSET_CSV)
asset_mapping = dict(asset_df[['Asset_ID', 'Asset_Name']].to_pandas().values)
asset_reverse_mapping = {v:k for k,v in asset_mapping.items()}
display(asset_df.head())

print("\n... TRAIN DATAFRAME ..\n")
TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")
train_df = cudf.read_csv(TRAIN_CSV)
train_df["Count"] = train_df["Count"].astype("int32")
train_df["asset_name"] = train_df["Asset_ID"].map(asset_mapping)
train_df['timestamp'] = cudf.to_datetime(train_df['timestamp'], unit='s')
train_df.columns = [_c.lower() for _c in train_df.columns]
display(train_df.head())

print("\n... SUPPLEMENTAL TRAIN DATAFRAME ..\n")
SUPP_TRAIN_CSV = os.path.join(DATA_DIR, "supplemental_train.csv")
supp_train_df = cudf.read_csv(SUPP_TRAIN_CSV)
supp_train_df["Count"] = supp_train_df["Count"].astype("int32")
supp_train_df["asset_name"] = supp_train_df["Asset_ID"].map(asset_mapping)
supp_train_df['timestamp'] = cudf.to_datetime(supp_train_df['timestamp'], unit='s')
supp_train_df.columns = [_c.lower() for _c in supp_train_df.columns]
display(supp_train_df.head())

print("\n... EXAMPLE TEST DATAFRAME ..\n")
EX_TEST_CSV = os.path.join(DATA_DIR, "example_test.csv")
ex_test_df = cudf.read_csv(EX_TEST_CSV)
ex_test_df["Count"] = ex_test_df["Count"].astype("int32")
ex_test_df["asset_name"] = ex_test_df["Asset_ID"].map(asset_mapping)
ex_test_df['timestamp'] = cudf.to_datetime(ex_test_df['timestamp'], unit='s')
ex_test_df.columns = [_c.lower() for _c in ex_test_df.columns]
display(ex_test_df.head())

print("\n... EXAMPLE SAMPLE SUBMISSION DATAFRAME ..\n")
EX_SS_CSV = os.path.join(DATA_DIR, "example_sample_submission.csv")
ex_ss_df = cudf.read_csv(EX_SS_CSV)
display(ex_ss_df.head())

# Set Other Variables
print("\n... SETTING OTHER VARIABLES ..\n")

# Create a new contest
env = gresearch_crypto.make_env()

# You can only iterate through a result from `env.iter_test()` once
# so be careful not to lose it once you start iterating.
iter_test = env.iter_test()
print("\n\n... BASIC DATA SETUP FINISHING ...\n")

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #fb806f; background-color: #ffffff;">2.4 LEVERAGING XLA OPTIMIZATIONS</h3>

---


**XLA** (Accelerated Linear Algebra) is a domain-specific compiler for linear algebra that can accelerate TensorFlow models with potentially no source code changes. **The results are improvements in speed and memory usage**.

<br>

When a TensorFlow program is run, all of the operations are executed individually by the TensorFlow executor. Each TensorFlow operation has a precompiled GPU/TPU kernel implementation that the executor dispatches to.

XLA provides us with an alternative mode of running models: it compiles the TensorFlow graph into a sequence of computation kernels generated specifically for the given model. Because these kernels are unique to the model, they can exploit model-specific information for optimization.<br><br>

<div class="alert alert-block alert-danger" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">🛑 &nbsp; WARNING:</b><br><br>- XLA can not currently compile functions where dimensions are not inferrable: that is, if it's not possible to infer the dimensions of all tensors without running the entire computation<br>
</div>

<div class="alert alert-block alert-info" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">📌 &nbsp; NOTE:</b><br><br>- XLA compilation is only applied to code that is compiled into a graph (in <b>TF2</b> that's only a code inside <b><code>tf.function</code></b>).<br>- The <b><code>jit_compile</code></b> API has must-compile semantics, i.e. either the entire function is compiled with XLA, or an <b><code>errors.InvalidArgumentError</code></b> exception is thrown)
</div>

<div class="alert alert-block alert-info" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">📖 &nbsp; REFERENCE:</b><br><br>    - <a href="https://www.tensorflow.org/xla"><b>XLA: Optimizing Compiler for Machine Learning</b></a><br>
</div>

In [ ]:
print(f"\n... XLA OPTIMIZATIONS STARTING ...\n")

print(f"\n... CONFIGURE JIT (JUST IN TIME) COMPILATION ...\n")
# enable XLA optmizations (10% speedup when using @tf.function calls)
tf.config.optimizer.set_jit(True)

print(f"\n... XLA OPTIMIZATIONS COMPLETED ...\n")

<br>


<a id="helper_functions"></a>


<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: #fb806f; background-color: #ffffff;" id="helper_functions">
    3&nbsp;&nbsp;HELPER FUNCTION & CLASSES&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a>
</h1>

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #fb806f; background-color: #ffffff;">3.1 GENERAL HELPER FUNCTIONS</h3>

---

In [ ]:
def flatten_l_o_l(nested_list):
    """ Flatten a list of lists """
    return [item for sublist in nested_list for item in sublist]

def reduce_crypto_df(df, asset_id=None, asset_name=None, dt_start='2021-04-14 05:45:00', dt_end='2021-04-14 07:00:00', dt_interval=1):
    """ Useful function to return the subset of a dataframe with desired information 
    
    Args:
        df (CuDF.DataFrame)
        asset_id (int, optional): Integer mapping for respective Cryptocurrency
            - 0 through 13 (14 total currencies as shown below)
        asset_name (str, optional): Name of the asset (cryptocurrency)
            - [Binance Coin|Bitcoin|Bitcoin Cash|Cardano]
            - [Dogecoin|EOS.IO|Ethereum|Ethereum Classic]
            - [IOTA|Litecoin|Maker|Monero|Stellar|TRON]
        dt_start (str, optional): The datetime to start the plot from (inclusive of this)
        dt_end (str, optional): The datetime to plot up until (non-inclusive of this) 
        dt_interval (int, optional): The number of minutes per displayed candlestick
        
    Returns:
        a CuDF.DataFrame containing the relevant stock information for the desired time period
    """
    
    if asset_id is None:
        if asset_name is None:
            raise ValueError("asset_id or asset_name must be provided")
        else:
            asset_id = asset_reverse_mapping[asset_name]
    else:
        asset_name = asset_mapping[asset_id]
    
    return df[df.asset_id==asset_id].set_index('timestamp').loc[dt_start:dt_end:dt_interval]


In [ ]:
# Let's look at the latest data (first half of September 2021)
DT_START = "2021-09-01 00:00:00"
DT_END = "2021-09-21 00:00:00"
DT_INTERVAL = 60 # every 2 hours

for _asset_id in train_df.asset_id.unique().to_pandas():
    sub_crypto_df = reduce_crypto_df(train_df, asset_id=_asset_id, dt_start=DT_START, dt_end=DT_END, dt_interval=DT_INTERVAL)
    qf = cf.QuantFig(sub_crypto_df.to_pandas(), columns={k.title():k for k in sub_crypto_df.columns}, name=asset_mapping[_asset_id])
    qf.iplot()